In [1]:
P10 = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6]
P8 = [6, 3, 7, 4, 8, 5, 10, 9]

In [2]:
def gen_subkeys(key):
    permuted = [key[i - 1] for i in P10]
    
    left_shift_1 = permuted[1:5]+permuted[:1]+permuted[6:]+permuted[5:6]
    
    key_1 = [left_shift_1[i - 1] for i in P8]
    
    left_shift_2 = left_shift_1[2:5]+left_shift_1[:2]+left_shift_1[7:]+left_shift_1[5:7]
    
    key_2 = [left_shift_2[i - 1] for i in P8]
    
    return key_1, key_2

In [3]:
IP = [2, 6, 3, 1, 4, 8, 5, 7]

In [4]:
IPI = [4, 1, 3, 5, 7, 2, 8, 6]

In [5]:
import numpy as np

In [6]:
S0 = np.array([1, 0, 3, 2, 3, 2, 1, 0, 0, 2, 1, 3, 3, 1, 3, 2]).reshape(4, 4)
S1 = np.array([0, 1, 2, 3, 2, 0, 1, 3, 3, 0, 1, 0, 2, 1, 0, 3]).reshape(4, 4)
EP = [4, 1, 2, 3, 2, 3, 4, 1]
P4 = [2, 4, 3, 1]

In [7]:
def f(text, key):
    expansion = [text[4:][i - 1] for i in EP]

    add_key = np.logical_xor(np.array(expansion).reshape(2, 4), np.array(key).reshape(2, 4))

    s_out_0 = np.binary_repr(S0[2 * add_key[0][0] + add_key[0][3]][2 * add_key[0][1] + add_key[0][2]], width = 2)
    s_out_1 = np.binary_repr(S1[2 * add_key[1][0] + add_key[1][3]][2 * add_key[1][1] + add_key[1][2]], width = 2)
    
    s_out = list(map(int, s_out_0)) + list(map(int, s_out_1))
    
    return np.concatenate((np.logical_xor(text[:4], [s_out[i - 1] for i in P4]), text[4:]))

In [8]:
def des(text, key_1, key_2):    
    permuted = [text[i - 1] for i in IP]
    
    f_1 = f(permuted, key_1)
    f_2 = f(np.concatenate((f_1[4:], f_1[:4])), key_2)
    
    return [f_2[i - 1] for i in IPI]

In [9]:
def encrypt(plaintext, key):
    key_1, key_2 = gen_subkeys(key)
    
    return des(plaintext, key_1, key_2)

def decrypt(ciphertext, key):
    key_1, key_2 = gen_subkeys(key)
    
    return des(ciphertext, key_2, key_1)

In [10]:
def trip_encrypt(plaintext, key_1, key_2):
    return encrypt(decrypt(encrypt(plaintext, key_1), key_2), key_1)

In [11]:
def trip_decrypt(ciphertext, key_1, key_2):
    return decrypt(encrypt(decrypt(ciphertext, key_1), key_2), key_1)

In [12]:
assert gen_subkeys([1, 0, 1, 0, 0, 0, 0, 0, 1, 0]) == ([1, 0, 1, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 0, 1, 1])

In [13]:
verification = '''0000000000 10101010 00010001
1110001110 10101010 11001010
1110001110 01010101 01110000
1111111111 10101010 00000100'''.split('\n')

tests = [[list(map(int, t)) for t in test.split()] for test in verification]

for key, plaintext, ciphertext in tests:
    assert encrypt(plaintext, key) == ciphertext
    assert decrypt(ciphertext, key) == plaintext

In [14]:
cxt_1 = '''010001110000000101000000110011011100101100000001011101000000000101101110010101110101011101101110010001110000000101000111101110100100111110001000010001110110111001001100101011111001011101101110011011101011101001001111101011110000100101001010100010000100111111001101100101110100111100110010000000010101011101101110100100000100111110101111010001111010111101110100011101000000000101001100000000010110111010111010100010000100011101101110010011001010111110010111000000011000100010010000'''

In [15]:
ctx_2 = '''000000011010011100110010110001100110010010100111110101111010011110011100011101000111010010011100000000011010011100000001100110011010000111011010000000011001110011101111011111100010010010011100100111001001100110100001011111101010000010110011110110101010000111000110001001001010000100100011101001110111010010011100010000011010000101111110000000010111111011010111110101111010011111101111101001111001110010011001110110100000000110011100111011110111111000100100101001111101101001000001'''

In [16]:
import string

found = False
found_key = None

for key in [list(map(int, np.binary_repr(i, width = 10))) for i in range(1024)]:
    if found:
        found_key = last_key
        
        break
        
    last_key = key
    
    for block in [list(map(int, cxt_1[i:i + 8])) for i in range(0, len(cxt_1), 8)]:
        dec = decrypt(block, key)
        
        if chr(int(''.join(map(str, dec)), 2)) not in string.printable:
            found = False
            
            break
            
        found = True

In [17]:
plaintext = ''

for block in [list(map(int, cxt_1[i:i + 8])) for i in range(0, len(cxt_1), 8)]:
    plaintext += chr(int(''.join(map(str, decrypt(block, found_key))), 2))

plaintext

'simplifieddesisnotsecureenoughtoprovideyousufficientsecurity'

In [18]:
found = False
found_key = None

p = 0

for key in [list(map(int, np.binary_repr(i, width = 20))) for i in range(pow(2, 20), 0, -1)]:
    if int(''.join(map(str, key)), 2) % pow(2, 13) == 0:
        p += 1
        
        print(p, "of", int(pow(2, 20 - 13)))
        
    if found:
        found_key = last_key
        
        break
        
    last_key = key
    
    for block in [list(map(int, cxt_1[i:i + 8])) for i in range(0, len(cxt_1), 8)]:
        dec = trip_decrypt(block, key[:10], key[10:])
        
        if chr(int(''.join(map(str, dec)), 2)) not in string.printable:
            found = False
            
            break
            
        found = True

1 of 128
2 of 128
3 of 128


In [19]:
plaintext = ''

for block in [list(map(int, cxt_1[i:i + 8])) for i in range(0, len(cxt_1), 8)]:
    plaintext += chr(int(''.join(map(str, trip_decrypt(block, found_key[:10], found_key[10:]))), 2))

plaintext

'simplifieddesisnotsecureenoughtoprovideyousufficientsecurity'